In [1]:
# !pip install openpyxl

In [2]:
import re
import numpy as np
import pandas as pd
from scipy import stats
from functools import reduce
import json

In [3]:
def create_gene_burden_table_helper(burden_df, annotations, maf, lf_samples_df, hgnc_dict):
    burden_df["gene"] = burden_df.gene.map(hgnc_dict)
    masked_burden_df = burden_df.loc[(burden_df.annotation.isin(annotations))&(burden_df.maf_max<=maf)].groupby("gene").agg({"samples": lambda x: set(",".join(x).split(","))}).reset_index()
    masked_burden_df = pd.concat([masked_burden_df, lf_samples_df])
    return masked_burden_df

def create_gene_burden_tables(burden_df, maf, lf_samples_df, hgnc_dict):
    masks = ["pLoF", "Missense_strict", "Missense_lenient"]
    annot_terms = [["lof"], ["lof", "missense_strict"], ["lof", "missense_strict", "missense_lenient"]]
    gene_burden_dict = dict(zip(masks, [create_gene_burden_table_helper(burden_df, at, maf, lf_samples_df, hgnc_dict) for at in annot_terms]))
    return gene_burden_dict

def get_samples_helper(combos, genotype_df, cohort_samples):
    if len(set(combos).intersection(set(genotype_df.gene.values))) == len(combos):
        samples_per_gene = genotype_df.loc[genotype_df.gene.isin(combos)].samples.values
        samples_per_combo = reduce(lambda a,b: set(a).intersection(set(b)), samples_per_gene)
        samples_per_combo = cohort_samples.intersection(samples_per_combo)
    else:
        samples_per_combo = []
    return samples_per_combo

def get_samples(ser, gene_burden_dict, pop_samples):
    gene, mask = ser.gene, ser.gene_mask
    gene_samples_df = gene_burden_dict[mask]
    
    combos = [gene]
    if "lf" in ser.index:
        lf = ser.lf
        combos.append(lf)
    samples = get_samples_helper(combos, gene_samples_df, pop_samples)
    return samples


def get_table_icd(gene_samples, nongene_samples, comorbid_samples, field):
    table = [
        [len(gene_samples.intersection(comorbid_samples)), len(gene_samples.difference(comorbid_samples))],
        [len(nongene_samples.intersection(comorbid_samples)), len(nongene_samples.difference(comorbid_samples))]
    ]
    df = pd.DataFrame(table, columns=[f"{field}", f"No {field}"], index=["Combo", "Non Combo"])
    return df


def get_table_cat(gene_samples, nongene_samples, cat_samples, field):
    table = [
        [len(gene_samples.intersection(cat_samples)), len(gene_samples.difference(cat_samples))],
        [len(nongene_samples.intersection(cat_samples)), len(nongene_samples.difference(cat_samples))]
    ]
    df = pd.DataFrame(table, columns=[f"{field}", f"No {field}"], index=["Combo", "Non Combo"])
    return df


def get_comorbidity_data_helper(gene_samples, pop_samples, pheno_df, comorbidity):
    comorbid_samples = set(pheno_df.loc[pheno_df[comorbidity]==1].sample_names)
    nongene_samples = pop_samples.difference(gene_samples)
    df = get_table_icd(gene_samples, nongene_samples, comorbid_samples, comorbidity)
    data_dict = dict(zip(
        [f"gene_{comorbidity}", f"gene_non{comorbidity}", f"nongene_{comorbidity}", f"nongene_non{comorbidity}"],
        [df.iloc[0,0], df.iloc[0,1], df.iloc[1,0], df.iloc[1,1]]
    ))
    return data_dict


def get_comorbidity_data(ser, gene_burden_dict, pop_samples, comorbidities):
    gene_samples = get_samples(ser, gene_burden_dict, pop_samples)
    comorbid_dict = dict()
    for comorbidity in comorbidities:
        cdict = get_comorbidity_data_helper(gene_samples, pop_samples, pheno_df, comorbidity)
        comorbid_dict.update(cdict)
    comorbid_dict["gene"] = ser.gene
    comorbid_dict["mask"] = ser.gene_mask
    return pd.Series(comorbid_dict)


def upload_file_to_project(filename, proj_dir):
    dxpy.upload_local_file(filename, folder=proj_dir, parents=True)
    print(f"*********{filename} uploaded!!*********")
    os.remove(filename)
    return



In [4]:
# get monogenic meta file
monogenic_meta_df = pd.read_excel("./monogenic_meta.xlsx")

# get gene burden file
gene_burden_df = pd.read_csv("/mnt/project/notebooks/wes/burden_preparation/data/ukb_burden.tsv.gz", sep="\t")
gnomad_df = pd.read_csv(
    "/mnt/project/notebooks/wes/burden_preparation/data/gnomad_annot.tsv.gz", sep="\t",
    usecols=["locus", "alleles", "maf_gnomad_popmax"]
)
gene_burden_df = gene_burden_df.merge(gnomad_df, on=["locus", "alleles"])
gene_burden_df["maf_max"] = gene_burden_df.apply(lambda ser: max(ser.maf, ser.maf_gnomad_popmax), axis=1)

# get pheno file
pheno_df = pd.read_csv("/mnt/project/notebooks/bmi/data/pheno.csv.gz", dtype={"sample_names": str})
comorbidities = ["cvd", "cad", "ht", "t1d", "t2d", "hf", "af", "pe", "vt", "avs", "grd", "cls", "ccs", "cd", "nfld", "koa", "ob"]

# prepare hgnc dict
with open('hgnc_gene_map.json', 'r') as f:
    # Load the JSON data into a Python dictionary
    hgnc_dict = json.load(f)
    

In [5]:
gene_burden_dict = create_gene_burden_tables(gene_burden_df, 0.001, pd.DataFrame(), hgnc_dict)
pop_samples = set(pheno_df.sample_names.astype(str))


In [6]:
monogenic_ukb_comorbid_df = monogenic_meta_df.apply(get_comorbidity_data, axis=1 , args=(gene_burden_dict, pop_samples, comorbidities,))


In [7]:
monogenic_ukb_comorbid_df.set_index(["gene", "mask"]).to_csv("./monogenic_ukb_comorbid.csv.gz")
